In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

**Intro**

Исходя из данных о местоположении и конструкции здания, ваша цель состоит в прогнозировании уровня ущерба, нанесенного зданиям в результате землетрясения Горхе в Непале в 2015-м году.

**Data Downloading and Transformation**

In [ ]:
train_df = pd.read_csv('/kaggle/input/richters-predictor-modeling-earthquake-damage/train_values.csv', index_col='building_id')
test_df = pd.read_csv('/kaggle/input/richters-predictor-modeling-earthquake-damage/test_values.csv',  index_col='building_id')

In [ ]:
target_df = pd.read_csv('/kaggle/input/richters-predictor-modeling-earthquake-damage/train_labels.csv', index_col='building_id')

In [ ]:
train_df.head()

Набор данных в основном состоит из информации о структуре зданий. Каждый ряд в наборе данных представляет конкретное здание в регионе, пострадавшем от землетрясения с названием Gorkha.

В этом наборе данных имеется 39 столбцов, где столбец `building_id` является уникальным и случайным идентификатором. Остальные 38 атрибутов описаны ниже. Категориальные переменные были обфусцированы случайными строчными символами. Появление одного и того же символа в разных столбцах не означает одинаковое исходное значение.

Набор данных содержит следующие атрибуты:

- **geo_level_1_id**, **geo_level_2_id**, **geo_level_3_id** (type: int): географический регион, в котором находится здание, от самого крупного (level 1) до наиболее точного подрегиона (level 3). Возможные значения: level 1: 0-30, level 2: 0-1427, level 3: 0-12567.
- **count_floors_pre_eq** (type: int): количество этажей в здании до землетрясения.
- **age** (тип: int): возраст здания в годах.
- **area_percentage** (type: int): нормализованная площадь основания здания.
- **height_percentage** (тип: int): нормализованная высота здания.
- **land_surface_condition** (type: categorical): состояние поверхности земли, на которой было построено здание. Возможные значения: n, o, t.
- **foundation_type** (type: categorical): тип фундамента, используемый при строительстве. Возможные значения: h, i, r, u, w.
- **roof_type** (type: categorical): тип крыши, используемой при строительстве. Возможные значения: n, q, x.
- **ground_floor_type** (type: categorical): тип первого этажа. Возможные значения: f, m, v, x, z.
- **other_floor_type** (type: categorical): тип конструкций, используемых на более высоких этажах (кроме кровли). Возможные значения: j, q, s, x.
- **position** (type: categorical): позиция здания. Возможные значения: j, o, s, t.
- **plan_configuration** (type: categorical): конфигурация плана здания. Возможные значения: a, c, d, f, m, n, o, q, s, u.
- **has_superstructure_adobe_mud** (type: binary): переменная-флаг, указывающая, была ли надстройка сделана из сырцового кирпича из глины и резаной соломы.
- **has_superstructure_mud_mortar_stone** (type: binary): переменная-флаг, которая указывает, была ли надстройка изготовлена из грязевого раствора и камня.
- **has_superstructure_stone_flag** (type: binary): переменная-флаг, которая указывает, была ли надстройка изготовлена из камня.
- **has_superstructure_cement_mortar_stone** (type: binary): переменная-флаг, которая указывает, была ли надстройка изготовлена из цементного раствора и камня.
- **has_superstructure_mud_mortar_brick** (type: binary): переменная-флаг, которая указывает, была ли надстройка изготовлена из грязевого раствора и кирпича.
- **has_superstructure_cement_mortar_brick** (type: binary): переменная-флаг, которая указывает, была ли надстройка изготовлена из цементного раствора и кирпича.
- **has_superstructure_timber** (type: binary): переменная-флаг, которая указывает, была ли надстройка сделана из дерева.
- **has_superstructure_bamboo** (type: binary): переменная-флаг, которая указывает, была ли надстройка сделана из бамбука.
- **has_superstructure_rc_non_engineered** (type: binary): переменная-флаг, которая указывает, была ли надстройка изготовлена из необработанного железобетона.
- **has_superstructure_rc_engineered** (type: binary): переменная-флаг, которая указывает, была ли надстройка изготовлена из инженерного железобетона.
- **has_superstructure_other** (type: binary): переменная-флаг, которая указывает, была ли надстройка изготовлена из какого-либо другого материала.
- **legal_ownership_status** (type: categorical): правовой статус собственности на землю, на которой было построено здание. Возможные значения: a, r, v, w.
- **count_families** (тип: int): количество семей, которые живут в здании.
- **has_secondary_use** (type: binary): переменная-флаг, которая указывает, использовалось ли здание для каких-либо дополнительных целей.
- **has_secondary_use_agriculture** (type: binary): переменная-флаг, которая указывает, использовалось ли здание в сельскохозяйственных целях.
- **has_secondary_use_hotel** (type: binary): переменная-флаг, которая указывает, использовалось ли здание как гостиница.
- **has_secondary_use_rental** (type: binary): переменная-флаг, которая указывает, использовалось ли здание для сдачи в аренду.
- **has_secondary_use_institution** (type: binary): переменная-флаг, которая указывает, использовалось ли здание в качестве местоположения какого-либо учреждения.
- **has_secondary_use_school** (type: binary): переменная-флаг, которая указывает, использовалось ли здание как школа.
- **has_secondary_use_industry** (type: binary): переменная-флаг, указывающая, использовалось ли здание в промышленных целях.
- **has_secondary_use_health_post** (type: binary): переменная-флаг, указывающая, использовалось ли здание в качестве медицинского пункта.
- **has_secondary_use_gov_office** (type: binary): переменная-флаг, которая указывает, использовалось ли здание для правительственного учреждения.
- **has_secondary_use_use_police** (type: binary): переменная-флаг, указывающая, использовалось ли здание в качестве полицейского участка.
- **has_secondary_use_other** (type: binary): переменная-флаг, которая указывает, использовалось ли здание для других целей.

In [ ]:
target_df.head()

Мы прогнозируем уровень ущерба (`damage_grade`) от 1 до 3. Уровень ущерба – это порядковая переменная, означающая, что порядок важен. Эту задачу можно рассматривать как классификацию или проблему порядковой регрессии. (Порядковая регрессия иногда описывается нечто среднее между классификацией и регрессией.)

Уровень ущерба:
- **1** – низкий урон
- **2** – среднее количество урона
- **3** – почти полное уничтожение

Чтобы измерить производительность алгоритмов, будем использовать метрику **F1**, которая уравновешивает точность и полноту классификатора. Традиционно, оценка F1 используется для оценки качества двоичного классификатора, но, поскольку у нас есть три возможные метки, мы будем использовать вариант, называемый **микро-усредненной оценкой F1** (micro averaged F1 score).

$$F1_{micro} = \frac{2 \cdot P_{micro} \cdot R_{micro}}{P_{micro} + R_{micro}},$$

где $P_{micro}=\frac {\sum_{k=1}^{3}TP_k} {\sum_{k=1}^{3}(TP_k+FP_k)}, R_{micro}=\frac {\sum_{k=1}^{3}TP_k} {\sum_{k=1}^{3}(TP_k+FN_k)}$

In [ ]:
cat_features = ['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id', 'land_surface_condition', 'foundation_type', 'roof_type', 
                    'ground_floor_type', 'other_floor_type', 'position', 'plan_configuration', 'legal_ownership_status']
num_features = ['count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage', 'count_families']

In [ ]:
idx = train_df.shape[0]

In [ ]:
data_df = pd.concat([train_df, test_df], sort=False)

In [ ]:
data_cat = pd.DataFrame(index = data_df.index, 
                  data = data_df, 
                  columns = cat_features)

In [ ]:
data_num = data_df.drop(columns = cat_features)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(data_cat)

In [ ]:
data_cat_encoded = enc.transform(data_cat)

In [ ]:
type(data_cat_encoded)

In [ ]:
type(data_num)

In [ ]:
from scipy.sparse import coo_matrix, hstack
data = hstack((data_cat_encoded,data_num))

In [ ]:
data = data.astype(dtype='float16')

In [ ]:
X_train = data.tocsr()[:idx]
X_test = data.tocsr()[idx:]

In [ ]:
y_train = target_df['damage_grade']

**Training the first model**

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(C=1.0, random_state=42, solver='liblinear', max_iter=500)
log_reg.fit(X_train, y_train)

In [ ]:
y_pred = log_reg.predict(X_valid)

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_valid, y_pred, average='micro')

In [ ]:
X_train = data.tocsr()[:idx]
y_train = target_df['damage_grade']
log_reg.fit(X_train, y_train)

In [ ]:
y_pred = log_reg.predict(X_test)

In [ ]:
predicted_df = pd.DataFrame(y_pred.astype(np.int8), index = test_df.index, columns=['damage_grade'])
predicted_df.to_csv('submit.csv')